In [13]:
import numpy as np

# Importing standard Qiskit libraries
from qiskit import *
from qiskit import QuantumCircuit, transpile, Aer, IBMQ
from qiskit.tools.jupyter import *
from qiskit.visualization import *
from ibm_quantum_widgets import *
from qiskit.providers.aer import QasmSimulator

# Loading your IBM Quantum account(s)
provider = IBMQ.load_account()

ibmqfactory.load_account:WARNING:2021-08-06 05:27:23,391: Credentials are already in use. The existing account in the session will be replaced.


In [14]:
# importing the least busy quantum computer

from qiskit.providers.ibmq import least_busy

In [15]:
# the CHSH game circuit


def CHSH(x,y,a0 = 0, a1 = np.pi/2, b0 =np.pi/4, b1 = -np.pi/4):
    '''
    Here x is the bit received by ALice and  y is the
    bit recieved by Bob, and a_0 is the angle used by
    Alice if she gets 0, a_1 is the angle used when she
    get 1, similarly for Bob applies b_0 angle if he gets
    0 and b_1 if he gets 1
    '''
    
    
    # make a quantum circuit
    
    circ= QuantumCircuit(2,2)
    
    # making a bell pair
    circ.h(0)
    circ.cx(0,1)
    
    # check for the bits recieved
    
    if (x == 0):
        circ.ry(a0,0)
    else:
        circ.ry(a1,0)
        
        
        
    if(y==0):
        circ.ry(b0,1)
    else:
        circ.ry(b1,1)
        
    # make the measurement
    
    circ.measure(range(2), range(2))
    
    return circ

In [16]:
def win_prob(backend, shots = 8192, a0=0, a1=np.pi/2, b0=np.pi/4, b1= -np.pi/4):
    '''
    Calculating the winning probability of Alice and Bob
    if they use the Quantum Strategy
    '''
    total = 0
    circuits = [CHSH(0,0,a0,a1,b0,b1), CHSH(0,1,a0,a1,b0,b1),CHSH(1,0,a0,a1,b0,b1),CHSH(1,1,a0,a1,b0,b1)] # all the four possiblities
    job = execute(circuits,backend=backend, shots = shots)
    
    for qc in circuits[0:3]:
        counts = job.result().get_counts(qc)
        if ('00' in counts):
            total += counts['00']
        if ('11' in counts):
            total += counts['11']
        # for the fourth 
        counts = job.result().get_counts(circuits[3])
        if('01' in counts):
            total += counts['01']
        if('10' in counts):
            total += counts['10']
            
        return total/(2*shots)
    
    

In [19]:
# running on Quantum Simulator
backend = Aer.get_backend('qasm_simulator')

print("The Winning Probability is",win_prob(backend))

The Winning Probability is 0.8538818359375


In [ ]:
# run on an actual Quantum Computer

provider = IBMQ.load_account()

backend = least_busy(provider.backends(operational = True, simulator = False, status_msg='active',
                                      filters=lambda x: x.configuration().n_qubits >1))
print("we are executing on ...", backend)

print("The Winning Probability is ",win_prob(backend))

ibmqfactory.load_account:WARNING:2021-08-06 05:30:02,196: Credentials are already in use. The existing account in the session will be replaced.


we are executing on ... ibmq_manila


Here We showed the Quantum Advantage of the CHSH Game